In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as stats

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Comments before starting
**Please this is my first ML model i a lot of time. Can you tell me what did I miss? Or what would you do differently**

**Thanks!**

# **EDA(Exploratory data analysis)**
**First things first let's dive in in the data and do the EDA(Exploratory data analysis). These are the steps we are going to perform**
* Remove the empty or missing values
* Remove outliers
* Do some aggregations
* Show the Mean,Meadian and Mode with the Std Deviation and variance (Just some columns)
* Remove the columns that won't help our ML model

In [ ]:
df = pd.read_csv("/kaggle/input/heart-disease-dataset-uci/HeartDiseaseTrain-Test.csv")
df.head()

In [ ]:
#Let's count all the NaN from the Df
df.isna().sum()

In [ ]:
#Data is complete so let's go and start showing some data
columns_to_plot = ['sex','chest_pain_type','fasting_blood_sugar','rest_ecg','exercise_induced_angina','slope','vessels_colored_by_flourosopy','thalassemia']
df_columns_plot = df[columns_to_plot]
for column in df_columns_plot.columns.values:
    print(column)
    v_counts = df_columns_plot[column].value_counts(normalize=True)
    print(v_counts)
    v_counts.plot.barh()
    plt.show()

In [ ]:
df_with_target_as_letters = df
#If Kaggle wouldn't show that target columns are only 0 and 1 I would check if data were corrupted
df_with_target_as_letters['target_letters'] = df_with_target_as_letters['target'].apply(lambda x: 'No heart disease' if x == 0 else 'Heart Disease')

In [ ]:
#Let's do the same with target column
#Data is complete so let's go and start showing some data
#columns_to_plot = ['sex','chest_pain_type','fasting_blood_sugar','rest_ecg','exercise_induced_angina','slope','vessels_colored_by_flourosopy','thalassemia']
df_columns_plot = df_with_target_as_letters[columns_to_plot]
for column in df_columns_plot.columns.values:
    sns.countplot(data=df_columns_plot,x=df_with_target_as_letters[column],hue=df_with_target_as_letters['target_letters'])
    plt.show()

In [ ]:
#Test_Shapiro-Wilk This is new for me. What I understood was that this test helps to know if data is normally
#distributed
columns_numbers = ['age','cholestoral','Max_heart_rate','resting_blood_pressure']
for column in columns_numbers:
    stat , p_value = stats.shapiro(df[column])
    if p_value > 0.05:
        print("Normally distributed")
    else:
        print("Not a normal distribution")
    sns.histplot(data=df, x=column,kde=True)
    plt.show()

In [ ]:
sns.relplot(data=df_with_target_as_letters, x="resting_blood_pressure", y="age", hue="target_letters", col="fasting_blood_sugar",row = 'sex')
plt.show()
sns.relplot(data=df_with_target_as_letters, x="cholestoral", y="age", hue="target_letters", col="rest_ecg",row = 'sex')
plt.show()

In [ ]:
#Something interesting was found. Most Heart Deseases come when the fasting blood sugar is lower than 120 mg/ml. And Females tend to have more heart problems in this conditions.
dict_column_value = {
    'fasting_blood_sugar':'Lower than 120 mg/ml',
    'rest_ecg':['ST-T wave abnormality','Normal']
    
}

def column_show_male_female_info(column,value):
    df_key_val = df.loc[df[key] == value]
    df_key_val_f = df_key_val.loc[df_key_val['sex'] == 'Female']
    df_key_val_m = df_key_val.loc[df_key_val['sex'] == 'Male']
    v_counts_f = df_key_val_f['target_letters'].value_counts(normalize=True)
    v_counts_m = df_key_val_m['target_letters'].value_counts(normalize=True)
    return v_counts_f,v_counts_m

for key in dict_column_value:
    if isinstance(dict_column_value.get(key),list):
        for value in dict_column_value.get(key):
            print('column:',key,'value:',value)
            v_counts_f,v_counts_m = column_show_male_female_info(key,value)
            print('Female \n',v_counts_f)
            print('Male \n',v_counts_m)
    else:
        print('column:',key,'value:',dict_column_value.get(key))
        v_counts_f,v_counts_m = column_show_male_female_info(key,dict_column_value.get(key))
        print('Female \n',v_counts_f)
        print('Male \n',v_counts_m)
        



In [ ]:
#Preparing data
del df["target_letters"]
y = df['target']
del df["target"]
#To transform data from Categories to numbers
X = pd.get_dummies(df,columns=["sex", "chest_pain_type","fasting_blood_sugar","rest_ecg","exercise_induced_angina","slope","vessels_colored_by_flourosopy","thalassemia"], prefix=["sex", "chest_pain","fasting","ecg","angina","slope","vessels","tha"])

In [ ]:
#Let's dive in doing the ML model
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,recall_score,precision_score,confusion_matrix,plot_roc_curve,classification_report


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=2)

In [ ]:
def check_model_results(model,X_train, X_test, y_train, y_test):
    y_pred = model.fit(X_train, y_train).predict(X_test)
    #Confussion Matrix on 
    print(confusion_matrix(y_test, y_pred))
    # Accuracy
    print(accuracy_score(y_test, y_pred))
    # Recall
    print(recall_score(y_test, y_pred, average=None))
    # Precision
    print(precision_score(y_test, y_pred, average=None))
    print(classification_report(y_test,y_pred))
    print(plot_roc_curve(model,X_test,y_test))
    
    

In [ ]:
models =  {
    'NaiveBayes':GaussianNB(),
    'RandomForest':RandomForestClassifier(n_estimators = 110),
    'SupportVectorMachine':SVC(),
    'DecisionTree':DecisionTreeClassifier(),
    'KNeighboors':KNeighborsClassifier(1)
}
for key,model in models.items():
    print(key)
    check_model_results(model,X_train, X_test, y_train, y_test)

# **Models Results**
**These three models gave us more than 90% accuracy:**
* KNeighboors
* Decision Tree
* Random Forest

**I hugely appreciate if you tell me what would you do differently or what pieces of the process did I miss.**

**Thanks!**